In [75]:
import pandas as pd
import json
import os
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
%matplotlib inline
import hvplot.pandas

# Importing Housing Inventory Data

In [76]:
# Reading Housing List prices from `Resources/RDC_Inventory_Core_Metrics_County_History.csv`
house_listing_df = pd.read_csv('Resources/RDC_Inventory_Core_Metrics_County_History.csv')

print('Dataset has ',house_listing_df.shape[0],' records and ',house_listing_df.shape[1], ' columns' ,'\n')
house_listing_df.head(2)

Dataset has  204416  records and  40  columns 



,month_date_yyyymm,county_fips,county_name,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,quality_flag
0,202112,1055,"etowah, al",189900.0,-0.0491,0.0597,161,-0.0301,-0.3235,48.5,...,229683.282383,-0.0208,-0.0776,179,-0.0165,-0.3115,0.111801,0.0154,0.0194,0
1,202112,18129,"posey, in",167400.0,0.1017,0.0321,20,-0.2857,0.0000,47.5,...,201440.666667,0.0611,-0.0032,22,-0.2667,0.0476,0.100000,0.0286,0.0500,0


### Correcting the Date column format and County_name

In [77]:
# Changing the date column format to yyyy-mm-dd
house_listing_df['month_date_yyyymm'] = pd.to_datetime(house_listing_df['month_date_yyyymm'], format='%Y%m')

# Renaming the column `month_date_yyyymm` to `Date`
house_listing_df.rename(columns={'month_date_yyyymm':'Date'}, inplace=True)

# Creating new temporary Dataframe with split values of the column
temp_county_state_df = house_listing_df['county_name'].str.split(",", n =1 , expand = True)

#Re assigning county_name from the temporary Dataframe
house_listing_df['county_name'] = temp_county_state_df[0].str.strip()

#Creating a new column for State
house_listing_df['state'] = temp_county_state_df[1].str.strip()

# Selecting Data for Washington State
wa_house_listing_df = house_listing_df[house_listing_df['state'] == 'wa']
wa_house_listing_df['county_name'] = wa_house_listing_df['county_name'].str.title()

# Sorting Records by Date and County Name
wa_house_listing_df = wa_house_listing_df.sort_values(['Date' , 'county_name'], ascending = (True, True)).reset_index(drop = True)

wa_house_listing_df.head()

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Date,county_fips,county_name,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,quality_flag,state
0,2016-07-01,53001,Adams,125000.0,NaN,NaN,34,NaN,NaN,87.0,...,NaN,NaN,36,NaN,NaN,0.058824,NaN,NaN,0,wa
1,2016-07-01,53003,Asotin,220000.0,NaN,NaN,120,NaN,NaN,61.0,...,NaN,NaN,120,NaN,NaN,NaN,NaN,NaN,0,wa
2,2016-07-01,53005,Benton,286500.0,NaN,NaN,629,NaN,NaN,42.0,...,NaN,NaN,629,NaN,NaN,0.000000,NaN,NaN,0,wa
3,2016-07-01,53007,Chelan,394000.0,NaN,NaN,502,NaN,NaN,60.0,...,NaN,NaN,510,NaN,NaN,0.015936,NaN,NaN,0,wa
4,2016-07-01,53009,Clallam,325000.0,NaN,NaN,596,NaN,NaN,66.0,...,NaN,NaN,596,NaN,NaN,NaN,NaN,NaN,0,wa


In [78]:
#Filter the house_prices_df columns
columns = ["Date", "county_name", "median_listing_price", "active_listing_count", "average_listing_price", "total_listing_count"]
wa_house_listing_df = wa_house_listing_df [columns]
wa_house_listing_df = wa_house_listing_df.dropna()

wa_house_listing_df

,Date,county_name,median_listing_price,active_listing_count,average_listing_price,total_listing_count
0,2016-07-01,Adams,125000.0,34,148215.739130,36
1,2016-07-01,Asotin,220000.0,120,242104.607692,120
2,2016-07-01,Benton,286500.0,629,323683.533619,629
3,2016-07-01,Chelan,394000.0,502,527891.732448,510
4,2016-07-01,Clallam,325000.0,596,405908.950000,596
...,...,...,...,...,...,...
2568,2021-12-01,Wahkiakum,442500.0,10,436552.366667,13
2569,2021-12-01,Walla Walla,492250.0,45,738237.223197,103
2570,2021-12-01,Whatcom,599900.0,154,823382.326556,424
2571,2021-12-01,Whitman,350000.0,51,454680.474557,75


In [79]:
#Creating a data that sorts the listing price
wa_house_listing_df = (wa_house_listing_df [["Date", "county_name", "median_listing_price", "average_listing_price"]].groupby (['Date', 'county_name']).mean())

wa_house_listing_df                            

median_listing_price  average_listing_price
Date       county_name                                             
2016-07-01 Adams                    125000.0          148215.739130
           Asotin                   220000.0          242104.607692
           Benton                   286500.0          323683.533619
           Chelan                   394000.0          527891.732448
           Clallam                  325000.0          405908.950000
...                                      ...                    ...
2021-12-01 Wahkiakum                442500.0          436552.366667
           Walla Walla              492250.0          738237.223197
           Whatcom                  599900.0          823382.326556
           Whitman                  350000.0          454680.474557
           Yakima                   384750.0          430637.974253

[2573 rows x 2 columns]

In [88]:
#Create a bar plot of the averaeg listing price by date
wa_house_listing_df.hvplot.bar(
    x='Date',
    y='average_listing_price',
    groupby='county_name',
    title='Average Listing Price by Date',
    rot=90).opts(yformatter='%.0f')

:DynamicMap   [county_name]
   :Bars   [Date]   (average_listing_price)

In [89]:
#Create a bar plot of the average listing price by county
wa_house_listing_df.hvplot.bar(
    x='county_name',
    y='average_listing_price',
    title='Average Listing Price by County',
    rot=50).opts(yformatter='%.0f')

:Bars   [county_name]   (average_listing_price)

# Function to define Custom Rental DataFrame Transform

In [119]:
def custom_transform(rental_dataframe):
    
    # Getting Date Column Names seperately to create a seperate column for Date
    dates = rental_dataframe.columns[3:]
    
    # Creating new Series to store the type and value of Rental data(eg : City/County)
    col_name = rental_dataframe.columns[0]
    col_values = pd.Series(rental_dataframe.iloc[:,0])
    
    # Creating new DataFrame `col_details` for col_name(City_Name/County_Name) and `FIPS_Code`
    col_details = rental_dataframe.loc[:,[col_name , 'FIPS_Code']]
    
    # Creating a new DataFrame with values as the cross product of `dates` list and `col_values` list
    cross_product_df =  pd.MultiIndex.from_product([dates, col_values], names = ['Date', col_name])
    cross_product_df = pd.DataFrame(index = cross_product_df).reset_index()
    
    # Merging the new DataFrame with `col_details` DataFrame
    result_df = pd.merge(left = cross_product_df, right=col_details, left_on=col_name, right_on=col_name)
    result_df = result_df.sort_values(by=['Date',col_name] , ignore_index = True)
    
    
    rental_dataframe.set_index('FIPS_Code' , inplace=True)
    rental_dataframe = rental_dataframe.replace('     NA', np.NaN)
    
    for index, row in result_df.iterrows():
        date_col = row['Date']
        FIPS_Code = row['FIPS_Code']
        result_df.at[index, 'Average_Rental_Price'] = rental_dataframe.loc[FIPS_Code,date_col]
        
    result_df['Date'] = pd.to_datetime(result_df['Date'], format='%Y_%m')
    return result_df
    

# Importing Rental Data

In [120]:
# Reading Housing List prices from `Resources/RDC_Inventory_Core_Metrics_County_History.csv`
rental_county_df = pd.read_csv("Resources/rental_growth_county.csv")
rental_city_df = pd.read_csv("Resources/rental_growth_city.csv")

rental_county_df['County_Name'] = (rental_county_df['County_Name'].str.split(" ", n =1 , expand = True))[0]
rental_city_df['City_Name'] = (rental_city_df['City_Name'].str.split(",", n =1 , expand = True))[0]

display(rental_county_df.head())
display(rental_city_df.head())


,County_Name,FIPS_Code,Population,2017_01,2017_02,2017_03,2017_04,2017_05,2017_06,2017_07,...,2021_03,2021_04,2021_05,2021_06,2021_07,2021_08,2021_09,2021_10,2021_11,2021_12
0,Benton,53005,194168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Clark,53011,465384,1294.0,1305.0,1318.0,1328.0,1333.0,1339.0,1363.0,...,1531.0,1560.0,1595.0,1632.0,1673.0,1714.0,1737.0,1747.0,1741.0,1743.0
2,Grant,53025,94860,961.0,1009.0,1032.0,1032.0,982.0,974.0,958.0,...,980.0,1009.0,1033.0,1067.0,1091.0,1110.0,1137.0,1130.0,1121.0,1094.0
3,King,53033,2163257,1672.0,1700.0,1730.0,1760.0,1796.0,1817.0,1833.0,...,1665.0,1711.0,1767.0,1832.0,1905.0,1972.0,2015.0,2008.0,1973.0,1928.0
4,Kitsap,53035,262475,1368.0,1360.0,1355.0,1364.0,1396.0,1431.0,1457.0,...,1708.0,1778.0,1844.0,1920.0,2019.0,2120.0,2198.0,2177.0,2126.0,2049.0


,City_Name,FIPS_Code,Population,2017_01,2017_02,2017_03,2017_04,2017_05,2017_06,2017_07,...,2021_03,2021_04,2021_05,2021_06,2021_07,2021_08,2021_09,2021_10,2021_11,2021_12
0,Auburn,5303180,79110,1278,1252,1232,1261,1314,1348,1373,...,1512,1555,1611,1658,1687,1712,1749,1750,1730,1679
1,Bellevue,5305210,142242,2063,2099,2158,2201,2224,2240,2261,...,2143,2192,2255,2342,2441,2523,2575,2550,2494,2445
2,Bothell,5307380,44994,1624,1643,1728,1787,1854,1875,1916,...,1972,2023,2063,2086,2143,2216,2290,2304,2262,2240
3,Everett,5322640,108941,1223,1220,1252,1262,1292,1325,1347,...,1474,1495,1533,1576,1620,1666,1694,1716,1705,1690
4,Federal Way,5323515,96110,1344,1370,1394,1407,1414,1428,1456,...,1614,1646,1694,1730,1767,1798,1832,1836,1840,1834


In [121]:
# Calling `custom_transform` function on `rental_county_df` DataFrame and `rental_city_df` DataFrame
rental_county_df = custom_transform(rental_county_df)
rental_city_df = custom_transform(rental_city_df)

# Displaying the transformed Rental DataFrames.
display(rental_county_df.head())
display(rental_city_df.head())


,Date,County_Name,FIPS_Code,Average_Rental_Price
0,2017-01-01,Benton,53005,NaN
1,2017-01-01,Clark,53011,1294.0
2,2017-01-01,Grant,53025,961.0
3,2017-01-01,King,53033,1672.0
4,2017-01-01,Kitsap,53035,1368.0


,Date,City_Name,FIPS_Code,Average_Rental_Price
0,2017-01-01,Auburn,5303180,1278
1,2017-01-01,Bellevue,5305210,2063
2,2017-01-01,Bothell,5307380,1624
3,2017-01-01,Everett,5322640,1223
4,2017-01-01,Federal Way,5323515,1344


In [122]:
#Filter the columns and drop any unknown values for the county data
columns = ['Date','County_Name', 'Average_Rental_Price']
rental_county_df = rental_county_df [columns]
rental_county_df = rental_county_df.dropna()

rental_county_df

,Date,County_Name,Average_Rental_Price
1,2017-01-01,Clark,1294.0
2,2017-01-01,Grant,961.0
3,2017-01-01,King,1672.0
4,2017-01-01,Kitsap,1368.0
5,2017-01-01,Pierce,1218.0
...,...,...,...
535,2021-12-01,Kitsap,2049.0
536,2021-12-01,Pierce,1764.0
537,2021-12-01,Snohomish,1922.0
538,2021-12-01,Spokane,1275.0


In [123]:
#Filter the columns and drop any unknown values for the city data
columns = ['Date','City_Name', 'Average_Rental_Price']
rental_city_df = rental_city_df [columns]
rental_city_df = rental_city_df.dropna()

rental_city_df

,Date,City_Name,Average_Rental_Price
0,2017-01-01,Auburn,1278
1,2017-01-01,Bellevue,2063
2,2017-01-01,Bothell,1624
3,2017-01-01,Everett,1223
4,2017-01-01,Federal Way,1344
...,...,...,...
1555,2021-12-01,Spokane,1290
1556,2021-12-01,Spokane Valley,1310
1557,2021-12-01,Tacoma,1611
1558,2021-12-01,Vancouver,1678


In [124]:
#Groupby the county name and date for the average rental price using .mean()
rental_county_df = (rental_county_df [["Date", "County_Name", "Average_Rental_Price"]].groupby (['Date', 'County_Name']).mean())
rental_county_df


Average_Rental_Price
Date       County_Name                      
2017-01-01 Clark                      1294.0
           Grant                       961.0
           King                       1672.0
           Kitsap                     1368.0
           Pierce                     1218.0
...                                      ...
2021-12-01 Kitsap                     2049.0
           Pierce                     1764.0
           Snohomish                  1922.0
           Spokane                    1275.0
           Thurston                   1799.0

[480 rows x 1 columns]

In [125]:
#Groupby the city name and date for the average rental price using .mean()
rental_city_df = (rental_city_df [["Date", "City_Name", "Average_Rental_Price"]].groupby (['Date', 'City_Name']).mean())
rental_city_df

Average_Rental_Price
Date       City_Name                           
2017-01-01 Auburn                        1278.0
           Bellevue                      2063.0
           Bothell                       1624.0
           Everett                       1223.0
           Federal Way                   1344.0
...                                         ...
2021-12-01 Spokane                       1290.0
           Spokane Valley                1310.0
           Tacoma                        1611.0
           Vancouver                     1678.0
           Woodinville                   2520.0

[1497 rows x 1 columns]

In [130]:
#Create a line plot of the county's average rental price 
rental_county_df.hvplot.line(
    xlabel='Date',
    ylabel='Average Rental Price',
    groupby='County_Name',
    title='Average County Rental Price by Date',
).opts(yformatter='%.0f')

:DynamicMap   [County_Name]
   :Curve   [Date]   (Average_Rental_Price)

In [131]:
#Create a line plot of the city's average rental price 
rental_city_df.hvplot.line(
    xlabel='Date',
    ylabel='Average Rental Price',
    groupby='City_Name',
    title='Average City Rental Price by Date',
).opts(yformatter='%.0f')

:DynamicMap   [City_Name]
   :Curve   [Date]   (Average_Rental_Price)

# Function to define custom Dataframe Transform

In [134]:
def custom_dataframe_transform(building_df , df_column_type):
    # Getting Date Column Names seperately to create a seperate column for Date
    dates = building_df.columns[1:]
    
    # Creating new Series to store the type and value of Original data(eg : City/County)
    col_name = building_df.columns[0]
    col_values = pd.Series(building_df.iloc[:,0])
    
    # Creating a new DataFrame with values as the cross product of `dates` list and `col_values` list
    result_df =  pd.MultiIndex.from_product([dates, col_values], names = ['Date', col_name])
    result_df = pd.DataFrame(index = result_df).reset_index()
    
    building_df.set_index(col_name , inplace=True)
    
    for index, row in result_df.iterrows():
        col_identifier = row['Date']
        row_identifier = row[col_name]
        result_df.at[index, df_column_type] = building_df.loc[row_identifier,col_identifier]
        
    return result_df 

# Importing Building Completion Units Data

In [135]:
# Reading Building Completion Units Data from 
building_completion_county_df = pd.read_csv("Resources/building_completion_units_county_wise.csv" , thousands=',')
building_completion_city_df = pd.read_csv("Resources/building_completion_units_city_wise.csv" , thousands=',')

building_completion_county_df['COUNTIES'] = (building_completion_county_df['COUNTIES'].str.split(" ", n =1 , expand = True))[0]

#Checking the datatypes of columns
display(building_completion_county_df.head())
display(building_completion_city_df.head())

,COUNTIES,2016/17,2017/18,2018/19,2019/20,2020/21
0,Adams,146,40,43,93,120
1,Asotin,29,10,43,54,125
2,Benton,867,1400,1247,1431,1315
3,Chelan,414,600,369,686,582
4,Clallam,239,253,395,285,301


,CITIES,2016/17,2017/18,2018/19,2019/20,2020/21
0,Aberdeen,6,-12,4,17,19
1,Airway Heights,31,397,268,200,153
2,Anacortes,83,53,133,83,61
3,Arlington,19,210,199,362,458
4,Auburn,507,534,653,21,389


In [136]:
# Calling `custom_dataframe_transform` function on `building_completion_county_df` DataFrame and `building_completion_city_df` DataFrame

df_column_type = 'Units_Completed'
building_completion_county_df = custom_dataframe_transform(building_completion_county_df , df_column_type)
building_completion_city_df = custom_dataframe_transform(building_completion_city_df , df_column_type)

# Displaying the transformed Building Units DataFrames.
display(building_completion_county_df.head())
display(building_completion_city_df.head())

,Date,COUNTIES,Units_Completed
0,2016/17,Adams,146.0
1,2016/17,Asotin,29.0
2,2016/17,Benton,867.0
3,2016/17,Chelan,414.0
4,2016/17,Clallam,239.0


,Date,CITIES,Units_Completed
0,2016/17,Aberdeen,6.0
1,2016/17,Airway Heights,31.0
2,2016/17,Anacortes,83.0
3,2016/17,Arlington,19.0
4,2016/17,Auburn,507.0


In [138]:
#Create a bar plot of the units completed by the county 
building_completion_county_df.hvplot.bar(
    x='Date',
    y='Units_Completed',
    groupby='COUNTIES',
    title='County Units Completed by Date',
    rot=30
).opts(yformatter='%.0f')

:DynamicMap   [COUNTIES]
   :Bars   [Date]   (Units_Completed)

In [151]:
building_completion_county_df_sorted = building_completion_county_df.sort_values("Units_Completed")
plot_building_completion_county_df_sorted=building_completion_county_df_sorted.hvplot.bar(
    x='Date',
    y='Units_Completed',
    groupby='COUNTIES',
    title='County Units Completed by Date',
    rot=30
).opts(yformatter='%.0f')

In [152]:
#Create a bar plot of the units completed by the city 
building_completion_city_df.hvplot.bar(
    x='Date',
    y='Units_Completed',
    groupby='CITIES',
    title='City Units Completed by Date',
    rot=30
).opts(yformatter='%.0f')

:DynamicMap   [CITIES]
   :Bars   [Date]   (Units_Completed)

# Importing Building Permits Data

In [153]:
building_completion_city_df_sorted = building_completion_city_df.sort_values("Units_Completed")

plot_building_completion_city_df_sorted=building_completion_city_df_sorted.hvplot.bar(
    x='Date',
    y='Units_Completed',
    groupby='CITIES',
    title='City Units Completed by Date',
    rot=30
).opts(yformatter='%.0f')

In [157]:
plot_building_completion_city_df_sorted + plot_building_completion_county_df_sorted

:Layout
   .DynamicMap.I  :DynamicMap   [CITIES]
      :Bars   [Date]   (Units_Completed)
   .DynamicMap.II :DynamicMap   [COUNTIES]
      :Bars   [Date]   (Units_Completed)

In [115]:
building_permit_county_df = pd.read_csv("Resources/building_permit_county_wise.csv" , thousands=',')
building_permit_city_df = pd.read_csv("Resources/building_permit_city_wise.csv" , thousands=',')

building_permit_county_df['COUNTIES'] = (building_permit_county_df['COUNTIES'].str.split(" ", n =1 , expand = True))[0]
display(building_permit_county_df.head())
display(building_permit_city_df.head())

,COUNTIES,2016,2017,2018,2019,2020
0,Adams,31,31,47,73,99
1,Asotin,63,34,34,86,161
2,Benton,1389,1111,1285,1540,1345
3,Chelan,1750,442,590,606,670
4,Clallam,640,307,336,287,279


,CITIES,2016,2017,2018,2019,2020
0,Aberdeen,5,6,2,2,17
1,Airway Heights,33,468,184,56,87
2,Anacortes,75,91,127,66,108
3,Arlington,16,198,321,247,141
4,Auburn,534,139,50,23,83


In [116]:
# Calling `custom_dataframe_transform` function on `building_permit_county_df` DataFrame and `building_permit_city_df` DataFrame

df_column_type = 'Units_Permitted'
building_permit_county_df = custom_dataframe_transform(building_permit_county_df , df_column_type)
building_permit_city_df = custom_dataframe_transform(building_permit_city_df , df_column_type)

# Displaying the transformed Building Unit Permits DataFrames.
display(building_permit_county_df.head())
display(building_permit_city_df.head())

,Date,COUNTIES,Units_Permitted
0,2016,Adams,31.0
1,2016,Asotin,63.0
2,2016,Benton,1389.0
3,2016,Chelan,1750.0
4,2016,Clallam,640.0


,Date,CITIES,Units_Permitted
0,2016,Aberdeen,5.0
1,2016,Airway Heights,33.0
2,2016,Anacortes,75.0
3,2016,Arlington,16.0
4,2016,Auburn,534.0


In [1]:

import library(dplyr)
import library(ggplot2)

rental_county_df <- data.frame( state = c("washington"),
                  stringsAsFactors = FALSE )

rental_city_df <- data.frame(state = tolower(state.name), 
                             long = -120.740135,
                             lat = 47.751076,
                             stringsAsFactors = FALSE)
inner_join( df, by="state" )

ggplot(rental_city_df, aes(long, lat)) 
borders("state") 
geom_point(aes(color=freq,size=freq), show_guide=FALSE) 
theme(text=element_text(size=18)) 
scale_size(range=c(2,20)) 
scale_color_continuous(low="red",high="green") 
theme_bw()

SyntaxError: invalid syntax (1142030954.py, line 1)